In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse, r2_score,explained_variance_score
import sklearn.model_selection

from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split

from math import sqrt
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import split_scale



In [2]:
df = pd.read_csv('query_result.csv')

In [3]:
df

,id,propertylandusetypeid,calculatedfinishedsquarefeet,taxamount,taxvaluedollarcnt,bedroomcnt,bathroomcnt,transactiondate,fips
0,1772,261,1604.0,6089.82,498347.0,4,2.0,2017-06-30,6037
1,2028,261,2384.0,6679.55,549917.0,3,3.0,2017-06-01,6037
2,3273,261,1574.0,3876.31,235272.0,3,2.0,2017-06-01,6037
3,3429,261,1619.0,4206.15,340000.0,2,2.0,2017-06-29,6037
4,4505,261,2408.0,24353.42,2017254.0,2,3.0,2017-06-27,6037
...,...,...,...,...,...,...,...,...,...
15980,53499,261,2040.0,4532.87,321351.0,3,2.0,2017-06-30,6037
15981,53500,261,1292.0,813.11,32830.0,2,1.0,2017-06-30,6037
15982,53501,261,684.0,4495.39,324000.0,1,1.0,2017-06-30,6037
15983,53502,261,1536.0,4014.27,284770.0,4,2.0,2017-06-30,6037


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15985 entries, 0 to 15984
Data columns (total 9 columns):
id                              15985 non-null int64
propertylandusetypeid           15985 non-null int64
calculatedfinishedsquarefeet    15984 non-null float64
taxamount                       15958 non-null float64
taxvaluedollarcnt               15983 non-null float64
bedroomcnt                      15985 non-null int64
bathroomcnt                     15985 non-null float64
transactiondate                 15985 non-null object
fips                            15985 non-null int64
dtypes: float64(4), int64(4), object(1)
memory usage: 1.1+ MB


In [5]:
df.astype({'fips': 'str'}).dtypes

id                                int64
propertylandusetypeid             int64
calculatedfinishedsquarefeet    float64
taxamount                       float64
taxvaluedollarcnt               float64
bedroomcnt                        int64
bathroomcnt                     float64
transactiondate                  object
fips                             object
dtype: object

In [6]:
df.head()

,id,propertylandusetypeid,calculatedfinishedsquarefeet,taxamount,taxvaluedollarcnt,bedroomcnt,bathroomcnt,transactiondate,fips
0,1772,261,1604.0,6089.82,498347.0,4,2.0,2017-06-30,6037
1,2028,261,2384.0,6679.55,549917.0,3,3.0,2017-06-01,6037
2,3273,261,1574.0,3876.31,235272.0,3,2.0,2017-06-01,6037
3,3429,261,1619.0,4206.15,340000.0,2,2.0,2017-06-29,6037
4,4505,261,2408.0,24353.42,2017254.0,2,3.0,2017-06-27,6037


In [7]:
df = df.rename(columns={'calculatedfinishedsquarefeet': 'square_feet', 'taxamount':'taxes', 'fips':'FIPS', 'taxvaluedollarcnt':'home_value', 'bedroomcnt':'bedroom_count', 'bathroomcnt':'bathroom_count'})

In [8]:
df = df.drop(columns=['propertylandusetypeid', 'id', 'transactiondate'])
df

,square_feet,taxes,home_value,bedroom_count,bathroom_count,FIPS
0,1604.0,6089.82,498347.0,4,2.0,6037
1,2384.0,6679.55,549917.0,3,3.0,6037
2,1574.0,3876.31,235272.0,3,2.0,6037
3,1619.0,4206.15,340000.0,2,2.0,6037
4,2408.0,24353.42,2017254.0,2,3.0,6037
...,...,...,...,...,...,...
15980,2040.0,4532.87,321351.0,3,2.0,6037
15981,1292.0,813.11,32830.0,2,1.0,6037
15982,684.0,4495.39,324000.0,1,1.0,6037
15983,1536.0,4014.27,284770.0,4,2.0,6037


In [9]:
df.home_value.isnull().sum()

2

In [10]:
df[df['home_value'].isnull()].index.tolist()

[3970, 9994]

In [11]:
df = df.dropna()
df

,square_feet,taxes,home_value,bedroom_count,bathroom_count,FIPS
0,1604.0,6089.82,498347.0,4,2.0,6037
1,2384.0,6679.55,549917.0,3,3.0,6037
2,1574.0,3876.31,235272.0,3,2.0,6037
3,1619.0,4206.15,340000.0,2,2.0,6037
4,2408.0,24353.42,2017254.0,2,3.0,6037
...,...,...,...,...,...,...
15980,2040.0,4532.87,321351.0,3,2.0,6037
15981,1292.0,813.11,32830.0,2,1.0,6037
15982,684.0,4495.39,324000.0,1,1.0,6037
15983,1536.0,4014.27,284770.0,4,2.0,6037


In [63]:
assert (df['square_feet'] == 0).sum() < 1 , "There are zeros in this column. Handle them before proceeding"
assert (df['taxes'] == 0).sum() < 1 , "There are zeros in this column. Handle them before proceeding"
assert (df['home_value'] == 0).sum() < 1 , "There are zeros in this column. Handle them before proceeding"
assert (df['bedroom_count'] == 0).sum() < 1 , "There are zeros in this column. Handle them before proceeding"
assert (df['bathroom_count'] == 0).sum() < 1 , "There are zeros in this column. Handle them before proceeding"


In [12]:
df.isnull().sum()

square_feet       0
taxes             0
home_value        0
bedroom_count     0
bathroom_count    0
FIPS              0
dtype: int64

In [13]:
# fips = pd.read_clipboard()

In [14]:
# fips

In [15]:
# fips.to_csv('fips.csv', index = False) 

In [16]:
fips = pd.read_csv('fips.csv')

In [17]:
fips

,FIPS,Name,State
0,1001,Autauga,AL
1,1003,Baldwin,AL
2,1005,Barbour,AL
3,1007,Bibb,AL
4,1009,Blount,AL
...,...,...,...
3227,72151,Yabucoa,PR
3228,72153,Yauco,PR
3229,78010,St. Croix,VI
3230,78020,St. John,VI


In [18]:
fips = fips.rename(columns={'Name': 'County'})

In [19]:
df = df.merge(fips, left_on = 'FIPS', right_on = 'FIPS')

In [20]:
df

,square_feet,taxes,home_value,bedroom_count,bathroom_count,FIPS,County,State
0,1604.0,6089.82,498347.0,4,2.0,6037,Los Angeles,CA
1,2384.0,6679.55,549917.0,3,3.0,6037,Los Angeles,CA
2,1574.0,3876.31,235272.0,3,2.0,6037,Los Angeles,CA
3,1619.0,4206.15,340000.0,2,2.0,6037,Los Angeles,CA
4,2408.0,24353.42,2017254.0,2,3.0,6037,Los Angeles,CA
...,...,...,...,...,...,...,...,...
15951,2399.0,4873.70,357653.0,5,3.0,6111,Ventura,CA
15952,1909.0,7451.16,582354.0,4,2.0,6111,Ventura,CA
15953,2387.0,7728.46,606747.0,4,3.0,6111,Ventura,CA
15954,3644.0,15161.94,1254000.0,6,4.5,6111,Ventura,CA


In [21]:
df["tax_rate"] = df["taxes"] / df["home_value"]
df

,square_feet,taxes,home_value,bedroom_count,bathroom_count,FIPS,County,State,tax_rate
0,1604.0,6089.82,498347.0,4,2.0,6037,Los Angeles,CA,0.012220
1,2384.0,6679.55,549917.0,3,3.0,6037,Los Angeles,CA,0.012146
2,1574.0,3876.31,235272.0,3,2.0,6037,Los Angeles,CA,0.016476
3,1619.0,4206.15,340000.0,2,2.0,6037,Los Angeles,CA,0.012371
4,2408.0,24353.42,2017254.0,2,3.0,6037,Los Angeles,CA,0.012073
...,...,...,...,...,...,...,...,...,...
15951,2399.0,4873.70,357653.0,5,3.0,6111,Ventura,CA,0.013627
15952,1909.0,7451.16,582354.0,4,2.0,6111,Ventura,CA,0.012795
15953,2387.0,7728.46,606747.0,4,3.0,6111,Ventura,CA,0.012738
15954,3644.0,15161.94,1254000.0,6,4.5,6111,Ventura,CA,0.012091


In [22]:
df.County.value_counts()

Los Angeles    12388
Orange          2780
Ventura          788
Name: County, dtype: int64

In [23]:
county_tax_rate = df[['County', 'tax_rate']]

In [24]:
county_tax_rate

,County,tax_rate
0,Los Angeles,0.012220
1,Los Angeles,0.012146
2,Los Angeles,0.016476
3,Los Angeles,0.012371
4,Los Angeles,0.012073
...,...,...
15951,Ventura,0.013627
15952,Ventura,0.012795
15953,Ventura,0.012738
15954,Ventura,0.012091


In [25]:
tax_rate_avg = pd.DataFrame(round(df.groupby("County").tax_rate.mean(),3)) * 100

In [26]:
def split_my_data(df, train_pct=0.70, seed=123):
    train, test = train_test_split(df, train_size=train_pct, random_state=seed)
    return train, test

In [27]:
train, test = split_my_data(df)

In [28]:
X_train = train[['square_feet', 'bedroom_count', 'bathroom_count']]
X_test = test[['square_feet', 'bedroom_count', 'bathroom_count']]
y_train = train[['home_value']]
y_test = test[['home_value']]

In [29]:
baseline_model = y_train.copy()

In [30]:
baseline_model.head()

,home_value
999,73472.0
1425,440039.0
10418,506112.0
8011,657562.0
12632,6396075.0


In [31]:
baseline_model = baseline_model.rename(columns={'home_value':'actual'})
baseline_model

,actual
999,73472.0
1425,440039.0
10418,506112.0
8011,657562.0
12632,6396075.0
...,...
12252,411175.0
1346,120764.0
11646,630000.0
15725,380797.0


In [32]:
baseline_model['yhat_bl'] = baseline_model['actual'].mean()
baseline_model.head()

,actual,yhat_bl
999,73472.0,461839.646163
1425,440039.0,461839.646163
10418,506112.0,461839.646163
8011,657562.0,461839.646163
12632,6396075.0,461839.646163


In [33]:
ols_model = ols(formula='home_value ~ bedroom_count + bathroom_count + square_feet', data=train).fit()

In [34]:
baseline_model['ols_prediction'] = ols_model.predict(X_train)


In [35]:
# pd.options.display.float_format = '{:.3f}'.format

In [36]:
baseline_model.head()

,actual,yhat_bl,ols_prediction
999,73472.0,461839.646163,2.821289e+05
1425,440039.0,461839.646163,6.094094e+05
10418,506112.0,461839.646163,6.455827e+05
8011,657562.0,461839.646163,8.737636e+05
12632,6396075.0,461839.646163,2.066465e+06


In [37]:
baseline_model['residuals'] = baseline_model.ols_prediction - baseline_model.actual

In [38]:
baseline_model

,actual,yhat_bl,ols_prediction,residuals
999,73472.0,461839.646163,2.821289e+05,2.086569e+05
1425,440039.0,461839.646163,6.094094e+05,1.693704e+05
10418,506112.0,461839.646163,6.455827e+05,1.394707e+05
8011,657562.0,461839.646163,8.737636e+05,2.162016e+05
12632,6396075.0,461839.646163,2.066465e+06,-4.329610e+06
...,...,...,...,...
12252,411175.0,461839.646163,2.099044e+05,-2.012706e+05
1346,120764.0,461839.646163,2.513504e+05,1.305864e+05
11646,630000.0,461839.646163,3.078964e+05,-3.221036e+05
15725,380797.0,461839.646163,3.889194e+05,8.122430e+03


In [39]:
baseline_model['residual_bl'] = baseline_model['yhat_bl'] - baseline_model['actual']
baseline_model

,actual,yhat_bl,ols_prediction,residuals,residual_bl
999,73472.0,461839.646163,2.821289e+05,2.086569e+05,3.883676e+05
1425,440039.0,461839.646163,6.094094e+05,1.693704e+05,2.180065e+04
10418,506112.0,461839.646163,6.455827e+05,1.394707e+05,-4.427235e+04
8011,657562.0,461839.646163,8.737636e+05,2.162016e+05,-1.957224e+05
12632,6396075.0,461839.646163,2.066465e+06,-4.329610e+06,-5.934235e+06
...,...,...,...,...,...
12252,411175.0,461839.646163,2.099044e+05,-2.012706e+05,5.066465e+04
1346,120764.0,461839.646163,2.513504e+05,1.305864e+05,3.410756e+05
11646,630000.0,461839.646163,3.078964e+05,-3.221036e+05,-1.681604e+05
15725,380797.0,461839.646163,3.889194e+05,8.122430e+03,8.104265e+04


In [40]:
mse_ols = mse(baseline_model.actual, baseline_model.ols_prediction)
mse_bl = mse(baseline_model.actual, baseline_model.yhat_bl)

mse_ols < mse_bl 


True

In [41]:
ols_r2 = round(ols_model.rsquared,3)
ols_r2

0.419

In [42]:
ols_p_value = ols_model.f_pvalue
ols_p_value

0.0

In [43]:
scaler, X_train_scaled, X_test_scaled = split_scale.min_max_scaler(X_train, X_test)

In [44]:
predictions = pd.DataFrame({'actual': train.home_value})

In [45]:
predictions

,actual
999,73472.0
1425,440039.0
10418,506112.0
8011,657562.0
12632,6396075.0
...,...
12252,411175.0
1346,120764.0
11646,630000.0
15725,380797.0


In [46]:
X, y = train[['bedroom_count' , 'bathroom_count', 'square_feet']], train.home_value

lm = sklearn.linear_model.LinearRegression().fit(X, y)
predictions['home_value ~ bedroom_count + bathroom_count + square_feet'] = lm.predict(X)

In [47]:
predictions

,actual,home_value ~ bedroom_count + bathroom_count + square_feet
999,73472.0,2.821289e+05
1425,440039.0,6.094094e+05
10418,506112.0,6.455827e+05
8011,657562.0,8.737636e+05
12632,6396075.0,2.066465e+06
...,...,...
12252,411175.0,2.099044e+05
1346,120764.0,2.513504e+05
11646,630000.0,3.078964e+05
15725,380797.0,3.889194e+05


In [48]:
predictions = predictions.rename(columns={'home_value ~ bedroom_count + bathroom_count + square_feet':'lm_prediction'})

In [49]:
predictions

,actual,lm_prediction
999,73472.0,2.821289e+05
1425,440039.0,6.094094e+05
10418,506112.0,6.455827e+05
8011,657562.0,8.737636e+05
12632,6396075.0,2.066465e+06
...,...,...
12252,411175.0,2.099044e+05
1346,120764.0,2.513504e+05
11646,630000.0,3.078964e+05
15725,380797.0,3.889194e+05
